In [11]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("/Users/Emily/documents/class_data/neural_network_charity_analysis/resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [12]:
application_df = application_df.drop(columns=["EIN", "NAME"])


In [13]:
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [14]:
count_apps = application_df.APPLICATION_TYPE.value_counts()
count_apps.head()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Determine which values to replace if counts are less than ...?
replace_application = list(count_apps[count_apps < 300].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [16]:
# Look at CLASSIFICATION value counts for binning
classify_count = application_df.CLASSIFICATION.value_counts()
classify_count.head()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [17]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classify_count[classify_count < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [18]:
category = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [19]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[category]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(category)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(category,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [21]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# attempt1
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_feat = len(X_train[0])
hidden_node1 =  50
hidden_node2 = 20
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, input_dim=input_feat, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                2200      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 3,241
Trainable params: 3,241
Non-trainable params: 0
_________________________________________________________________


In [24]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Train the model
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000,
    period=5)

fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/100
  928/25724 [>.............................] - ETA: 1:07 - loss: 0.6927 - accuracy: 0.5302
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1792/25724 [=>............................] - ETA: 36s - loss: 0.6742 - accuracy: 0.5787
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2848/25724 [==>...........................] - ETA: 23s - loss: 0.6527 - accuracy: 0.6173
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 3904/25724 [===>..........................] - ETA: 17s - loss: 0.6390 - accuracy: 0.6404
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5024/25724 [====>.........................] - ETA: 13s - loss: 0.6333 - accuracy: 0.6529
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6016/25724 [======>.......................] - ETA: 11s - loss: 0.6250 - accuracy: 0.6661
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6976/25724 [=======>......................] - ETA: 10s - loss: 0.6204 - accuracy:

 3584/25724 [===>..........................] - ETA: 4s - loss: 0.5530 - accuracy: 0.7358
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 4608/25724 [====>.........................] - ETA: 4s - loss: 0.5607 - accuracy: 0.7287
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 5760/25724 [=====>........................] - ETA: 3s - loss: 0.5637 - accuracy: 0.7259
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 6688/25724 [======>.......................] - ETA: 3s - loss: 0.5643 - accuracy: 0.7252
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 7680/25724 [=======>......................] - ETA: 3s - loss: 0.5638 - accuracy: 0.7254
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 8864/25724 [=========>....................] - ETA: 3s - loss: 0.5612 - accuracy: 0.7269
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 9920/25724 [==========>...................] - ETA: 3s - loss: 0.5633 - accuracy: 0.7256
Epoch 00003: saving model to checkp

 8608/25724 [=========>....................] - ETA: 2s - loss: 0.5549 - accuracy: 0.7275
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 9600/25724 [==========>...................] - ETA: 2s - loss: 0.5535 - accuracy: 0.7289
Epoch 00005: saving model to checkpoints/weights.05.hdf5
10720/25724 [===========>..................] - ETA: 2s - loss: 0.5552 - accuracy: 0.7278
Epoch 00005: saving model to checkpoints/weights.05.hdf5
11584/25724 [============>.................] - ETA: 2s - loss: 0.5552 - accuracy: 0.7276
Epoch 00005: saving model to checkpoints/weights.05.hdf5
12736/25724 [=============>................] - ETA: 2s - loss: 0.5544 - accuracy: 0.7284
Epoch 00005: saving model to checkpoints/weights.05.hdf5
13696/25724 [==============>...............] - ETA: 2s - loss: 0.5537 - accuracy: 0.7288
Epoch 00005: saving model to checkpoints/weights.05.hdf5
14784/25724 [================>.............] - ETA: 2s - loss: 0.5554 - accuracy: 0.7275
Epoch 00005: saving model to checkp

13184/25724 [==============>...............] - ETA: 2s - loss: 0.5570 - accuracy: 0.7266
Epoch 00007: saving model to checkpoints/weights.07.hdf5
14048/25724 [===============>..............] - ETA: 1s - loss: 0.5558 - accuracy: 0.7279
Epoch 00007: saving model to checkpoints/weights.07.hdf5
15392/25724 [================>.............] - ETA: 1s - loss: 0.5556 - accuracy: 0.7277
Epoch 00007: saving model to checkpoints/weights.07.hdf5
16192/25724 [=================>............] - ETA: 1s - loss: 0.5567 - accuracy: 0.7275
Epoch 00007: saving model to checkpoints/weights.07.hdf5
17184/25724 [===================>..........] - ETA: 1s - loss: 0.5569 - accuracy: 0.7271
Epoch 00007: saving model to checkpoints/weights.07.hdf5
18560/25724 [====================>.........] - ETA: 1s - loss: 0.5558 - accuracy: 0.7279
Epoch 00007: saving model to checkpoints/weights.07.hdf5
19424/25724 [=====================>........] - ETA: 0s - loss: 0.5551 - accuracy: 0.7288
Epoch 00007: saving model to checkp

18368/25724 [====================>.........] - ETA: 1s - loss: 0.5525 - accuracy: 0.7281
Epoch 00009: saving model to checkpoints/weights.09.hdf5
19328/25724 [=====================>........] - ETA: 1s - loss: 0.5518 - accuracy: 0.7289
Epoch 00009: saving model to checkpoints/weights.09.hdf5
20224/25724 [======================>.......] - ETA: 1s - loss: 0.5505 - accuracy: 0.7306
Epoch 00009: saving model to checkpoints/weights.09.hdf5
21280/25724 [=======================>......] - ETA: 1s - loss: 0.5511 - accuracy: 0.7302
Epoch 00009: saving model to checkpoints/weights.09.hdf5
22432/25724 [=========================>....] - ETA: 0s - loss: 0.5509 - accuracy: 0.7301
Epoch 00009: saving model to checkpoints/weights.09.hdf5
23296/25724 [==========================>...] - ETA: 0s - loss: 0.5516 - accuracy: 0.7290
Epoch 00009: saving model to checkpoints/weights.09.hdf5
24224/25724 [===========================>..] - ETA: 0s - loss: 0.5518 - accuracy: 0.7292
Epoch 00009: saving model to checkp

...

Epoch 00091: saving model to checkpoints/weights.91.hdf5
25724/25724 [==============================] - 6s 214us/sample - loss: 0.5363 - accuracy: 0.7394
Epoch 92/100
  256/25724 [..............................] - ETA: 6s - loss: 0.5597 - accuracy: 0.7383
Epoch 00092: saving model to checkpoints/weights.92.hdf5
 1632/25724 [>.............................] - ETA: 4s - loss: 0.5343 - accuracy: 0.7457
Epoch 00092: saving model to checkpoints/weights.92.hdf5
 2400/25724 [=>............................] - ETA: 5s - loss: 0.5276 - accuracy: 0.7475
Epoch 00092: saving model to checkpoints/weights.92.hdf5
 3584/25724 [===>..........................] - ETA: 4s - loss: 0.5324 - accuracy: 0.7450
Epoch 00092: saving model to checkpoints/weights.92.hdf5
 4608/25724 [====>.........................] - ETA: 4s - loss: 0.5357 - accuracy: 0.7428
Epoch 00092: saving model to checkpoints/weights.92.hdf5
 5536/25724 [=====>........................] - ETA: 4s - loss: 0.5376 - accuracy: 0.7399
Epoch 00092: s

 3200/25724 [==>...........................] - ETA: 4s - loss: 0.5379 - accuracy: 0.7394
Epoch 00094: saving model to checkpoints/weights.94.hdf5
 4192/25724 [===>..........................] - ETA: 4s - loss: 0.5405 - accuracy: 0.7350
Epoch 00094: saving model to checkpoints/weights.94.hdf5
 5216/25724 [=====>........................] - ETA: 3s - loss: 0.5411 - accuracy: 0.7339
Epoch 00094: saving model to checkpoints/weights.94.hdf5
 6432/25724 [======>.......................] - ETA: 3s - loss: 0.5426 - accuracy: 0.7307
Epoch 00094: saving model to checkpoints/weights.94.hdf5
 7488/25724 [=======>......................] - ETA: 3s - loss: 0.5402 - accuracy: 0.7350
Epoch 00094: saving model to checkpoints/weights.94.hdf5
 8416/25724 [========>.....................] - ETA: 3s - loss: 0.5401 - accuracy: 0.7365
Epoch 00094: saving model to checkpoints/weights.94.hdf5
 9184/25724 [=========>....................] - ETA: 3s - loss: 0.5382 - accuracy: 0.7378
Epoch 00094: saving model to checkp

 7136/25724 [=======>......................] - ETA: 3s - loss: 0.5313 - accuracy: 0.7393
Epoch 00096: saving model to checkpoints/weights.96.hdf5
 7936/25724 [========>.....................] - ETA: 3s - loss: 0.5316 - accuracy: 0.7407
Epoch 00096: saving model to checkpoints/weights.96.hdf5
 9088/25724 [=========>....................] - ETA: 3s - loss: 0.5362 - accuracy: 0.7378
Epoch 00096: saving model to checkpoints/weights.96.hdf5
10272/25724 [==========>...................] - ETA: 2s - loss: 0.5359 - accuracy: 0.7393
Epoch 00096: saving model to checkpoints/weights.96.hdf5
11328/25724 [============>.................] - ETA: 2s - loss: 0.5370 - accuracy: 0.7385
Epoch 00096: saving model to checkpoints/weights.96.hdf5
12288/25724 [=============>................] - ETA: 2s - loss: 0.5381 - accuracy: 0.7386
Epoch 00096: saving model to checkpoints/weights.96.hdf5
13024/25724 [==============>...............] - ETA: 2s - loss: 0.5386 - accuracy: 0.7383
Epoch 00096: saving model to checkp

11936/25724 [============>.................] - ETA: 2s - loss: 0.5329 - accuracy: 0.7438
Epoch 00098: saving model to checkpoints/weights.98.hdf5
12800/25724 [=============>................] - ETA: 2s - loss: 0.5329 - accuracy: 0.7441
Epoch 00098: saving model to checkpoints/weights.98.hdf5
13920/25724 [===============>..............] - ETA: 2s - loss: 0.5337 - accuracy: 0.7435
Epoch 00098: saving model to checkpoints/weights.98.hdf5
15040/25724 [================>.............] - ETA: 1s - loss: 0.5338 - accuracy: 0.7437
Epoch 00098: saving model to checkpoints/weights.98.hdf5
16096/25724 [=================>............] - ETA: 1s - loss: 0.5347 - accuracy: 0.7425
Epoch 00098: saving model to checkpoints/weights.98.hdf5
17088/25724 [==================>...........] - ETA: 1s - loss: 0.5348 - accuracy: 0.7420
Epoch 00098: saving model to checkpoints/weights.98.hdf5
18176/25724 [====================>.........] - ETA: 1s - loss: 0.5352 - accuracy: 0.7415
Epoch 00098: saving model to checkp

15904/25724 [=================>............] - ETA: 1s - loss: 0.5385 - accuracy: 0.7372
Epoch 00100: saving model to checkpoints/weights.100.hdf5
16800/25724 [==================>...........] - ETA: 1s - loss: 0.5383 - accuracy: 0.7377
Epoch 00100: saving model to checkpoints/weights.100.hdf5
18016/25724 [====================>.........] - ETA: 1s - loss: 0.5373 - accuracy: 0.7378
Epoch 00100: saving model to checkpoints/weights.100.hdf5
18912/25724 [=====================>........] - ETA: 1s - loss: 0.5368 - accuracy: 0.7391
Epoch 00100: saving model to checkpoints/weights.100.hdf5
19840/25724 [======================>.......] - ETA: 1s - loss: 0.5368 - accuracy: 0.7394
Epoch 00100: saving model to checkpoints/weights.100.hdf5
20960/25724 [=======================>......] - ETA: 0s - loss: 0.5368 - accuracy: 0.7398
Epoch 00100: saving model to checkpoints/weights.100.hdf5
21984/25724 [========================>.....] - ETA: 0s - loss: 0.5370 - accuracy: 0.7394
Epoch 00100: saving model to 

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")